In [ ]:
import naas
#Suchbegriffe
searchstrings = ["Indie","indie","Pop","pop","Gitarre","Gitarrist", "gitarre","gitarrist"]

import requests
## Create start values for while-loop
# Create empty list to store data
result = []
# Webseite-Anzahl
page_number = 1 
# give links_compare and links values to start while loop
links_compare = 1
links = 2
## Starting While-loop: Look for next page; if next page equals the current page; then stop loop
## Because the website goes automaically to the last page no matter which page you are typing in
while links_compare != links:
    # Die URL der Webseite
    url = f"https://www.musiker-sucht.de/band/plz:8/page:{page_number}"
    # Sende eine HTTP-GET-Anfrage an die URL und speichere die Antwort
    response = requests.get(url)
    html = response.text

    # Verwende die BeautifulSoup-Bibliothek, um den HTML-Code zu parsen und die Links auf der Seite zu finden
    from bs4 import BeautifulSoup

    soup = BeautifulSoup(html, "html.parser")
    links = soup.find_all("a")

    # Iteriere über jeden Link und öffne ihn mithilfe von requests
    for link in links:
        try:
            url = "https://musiker-sucht.de"+link["href"]
            if "/band/anzeige/" in url:
                #print("accepted URL")
                response = requests.get(url)
                soup = BeautifulSoup(response.text, "html.parser")
                entry = soup.find("p","classified-view message")
                entry = entry.get_text()
            # Verarbeite den HTML-Code der Seite, wie es für deine Zwecke erforderlich ist
            # und füge das Ergebnis der Liste hinzu
                if any(x in str(entry) for x in searchstrings):
                    #print("successfully added to list")
                    # Füge Link zur Anzeige hinzu
                    entry = str(entry)+" \n "+"Link zur Anzeige:"+url+" \n "
                    result.append(entry)
                #else:
                    #print("searchstring not found")
        except:
            #print("skip false URL")
            pass
    #print(str(len(result))+" elements are part of this list")
    # Speichere Anzahl der Ergebnisse als Zahl
    result_count = str(len(result))
    # Passe result-Objekt an, für Zeilenumbrüche
    result = str(result)
    result = result.replace("[","")
    result = result.replace("]","")
    result = result.replace("'","")
    result = result.replace('\\n','\n')

    ## Create html_compare to check whether the next page is new or the same as this one
    #increase page_number by 1 to check for next page
    page_number += 1
    print("looking for page number:" + str(page_number))
    # Die URL der Webseite
    url = f"https://www.musiker-sucht.de/band/plz:8/page:{page_number}"
    # Sende eine HTTP-GET-Anfrage an die URL und speichere die Antwort
    response = requests.get(url)
    html_compare = response.text
    soup_compare = BeautifulSoup(html_compare, "html.parser")
    links_compare = soup_compare.find_all("a")


from email.message import EmailMessage
import ssl
import smtplib

email_sender = "automated.code.az@t-online.de"
email_password = ""
email_receiver = ""

subject = f"{result_count} Kandidaten auf musiker-sucht.de"
body = """
Es wurde nach folgendem Suchtext gesucht im Postleitzahl-Bereich 8XXXX:
\n
{}
\n\n
{}
""".format(searchstrings,result)


em = EmailMessage()
em["From"] = email_sender
em["To"] = email_receiver
em["Subject"] = subject
em.set_content(body)

context = ssl.create_default_context()

with smtplib.SMTP_SSL("securesmtp.t-online.de",465,context = context)as smpt:
	smpt.login(email_sender,email_password)
	smpt.sendmail(email_sender,email_receiver, em.as_string())





naas.scheduler.add(recurrence = "0 21 * * *")